I ran this on Databricks PySpark

In [3]:
from datasets import load_dataset_builder
from psutil._common import bytes2human
from datasets import load_dataset

In [ ]:
def print_dataset_size_if_provided(*args, **kwargs):
  dataset_builder = load_dataset_builder(*args, **kwargs)

  if dataset_builder.info.download_size and dataset_builder.info.dataset_size:
    print(f'download_size={bytes2human(dataset_builder.info.download_size)}, dataset_size={bytes2human(dataset_builder.info.dataset_size)}')
  else:
    print('Dataset size is not provided by uploader')

print_dataset_size_if_provided("community-datasets/generics_kb", "generics_kb_best")

In [ ]:
ds = load_dataset("community-datasets/generics_kb", "generics_kb_best")

In [ ]:
df = ds["train"].to_pandas()

# Print the first 5 rows
print(df.columns)
print(df.iloc[0:5][["term", "generic_sentence"]])
print(len(df))
df = df[["term", "generic_sentence"]]

In [ ]:
# Lowercase the terms
df["term"] = df["term"].str.lower()
# Remove punctuation
df["generic_sentence"] = df["generic_sentence"].str.replace(r'[^\w\s]', '')

# Deduplicate
df = df.drop_duplicates()

In [ ]:
# Filter out rows with terms and sentences that are too short or too long
df = df[(df["term"].apply(len) >= 3) & (df["term"].apply(len) <= 20)]
df = df[(df["generic_sentence"].apply(len) >= 20) & (df["generic_sentence"].apply(len) <= 40)]
print(df.iloc[0:20][["term", "generic_sentence"]])
print(len(df))

In [ ]:
# Each term has multiple generic sentences. We will only keep the first few
df = df.groupby("term").head(3)
print(df.iloc[0:20][["term", "generic_sentence"]])
print(len(df))

In [ ]:
# Save the dataset
display(df)
# Saved as data/generic_sentences.csv
# Turns out the dataset is full of "Noun" is blablabla type of sentences. Not very useful

This I ran locally

In [56]:
ds = load_dataset("iastate/onestop_english")

df = ds["train"].to_pandas()

In [57]:
df = df[["text"]]
print(len(df))

# Split it by sentence and period
df = df["text"].str.split("\n", expand=True).stack().reset_index(level=1, drop=True).reset_index().rename(columns={0: "text"})
df = df["text"].str.split(".", expand=True).stack().reset_index(level=1, drop=True).reset_index().rename(columns={0: "text"})

# Only keep alphanumeric characters
df["text"] = df["text"].str.replace(r'[^a-zA-Z0-9\s]', '').str.strip()

print(len(df))

567
27412


C:\Users\Johnny Chen\AppData\Local\Temp\ipykernel_15524\1255900053.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"] = df["text"].str.replace(r'[^a-zA-Z0-9\s]', '').str.strip()


In [58]:
# Calculate the length of each sentence
df["length"] = df["text"].apply(len)
print(df["length"].describe())

count    27412.000000
mean        78.844119
std         69.536195
min          0.000000
25%          0.000000
50%         73.000000
75%        123.000000
max        811.000000
Name: length, dtype: float64


In [59]:
df = df[(df["length"] <= 100) & (df["length"] >= 20)]
print(len(df))

10107


In [60]:
from textnoisr import noise

def add_noise(text):
    augmenter = noise.CharNoiseAugmenter(noise_level=0.1)
    text = augmenter.add_noise(text)
    return text

In [61]:
df["noisy_text"] = df["text"].apply(add_noise)

In [62]:
print(df.iloc[0:1][["text", "noisy_text"]])

                                                text  \
1  These are the questions in a debate about the ...   

                                          noisy_text  
1  These are the questions in adebate aout the in...  


In [63]:
df = df[["text", "noisy_text"]].reset_index(drop=True)
df.to_csv("data/onestop_english_sentences.csv", index=False)